In [2]:
%pylab inline
import tensorflow as tf
from itertools import groupby
import pandas as pd
from collections import OrderedDict
from datetime import date
import pyodbc

Populating the interactive namespace from numpy and matplotlib


C:\Users\Vivi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Data Preprocessing (can be skipped after done once)

In [ ]:
records=pd.read_csv('C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/survstat/csv/cam_5y.csv',encoding='latin-1')
CoNew = pd.read_csv('C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/DieInfluenza/CoordBins.csv',encoding='latin-1')

In [ ]:
recordsX=records[['county','week','count','incidence']]
gb = recordsX.groupby(['week','county']).sum()

In [ ]:
gb.index[0]

In [ ]:
newDF = pd.DataFrame(columns=['week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty'])
x = 0
for i in gb.index:
    week = i[0]
    counties = i[1]
    counts = gb.values[x][0]
    incidences = gb.values[x][1]
    loc = CoNew[CoNew['county']==counties]
    if loc['X'].values>0:
        newDF=newDF.append({'week': str(week)[7:],'year':str(week)[:4],'county':str(counties),'count':str(counts),'incidence':str(incidences),'X':loc['X'].values,'Y':loc['Y'].values,'BinX':loc['BinX'].values,'BinY':loc['BinY'].values,'LocCounty':loc['county'].values}, ignore_index=True)    
    try:
        with open('StomachVirusInput.csv', 'a') as f:
            newDF.to_csv(f, header=False)
    except:
        pass
    newDF=pd.DataFrame(columns=['week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty'])
    x = x+1
#newDF.to_csv('StomachVirusInput.csv')

In [ ]:
newDF = pd.read_csv('StomachVirusInput.csv',encoding='latin-1')
newDF.columns = ['ID','week','year','county','count','incidence','X','Y','BinX','BinY','LocCounty']#.set_index('Unnamed: 0')
newDF = newDF.dropna()
newDF = newDF[['week','year','county','count','incidence','X','Y','BinX','BinY']].reset_index(drop=True)
newDF.to_csv('StomachVirusNew.csv')

In [ ]:
newDF = pd.read_csv('StomachVirusInput.csv',encoding='latin-1')
newDF

In [ ]:
test = newDF[newDF['year']==2001][newDF[newDF['year']==2001]['week']==2]['incidence']

In [ ]:
def FindLabels(data):
    LabelDF = pd.DataFrame(columns=['count','incidence','county'])
    for w in data.index:
        try:
            thisyear = data.iloc[w]['year']
            thisweek = data.iloc[w]['week']
            thisarea = data.iloc[w]['county']
            if int(thisweek)<52:
                nextweek = int(thisweek)+1
                nextyear = int(thisyear)
            else:
                nextweek = 1
                nextyear = int(thisyear)+1
            nextdf = data[data['year']==nextyear][data[data['year']==nextyear]['week']==nextweek]
            nextcount = np.sum(nextdf[nextdf['county']==thisarea]['count'])
            if nextcount>0:
                nextincidence = np.sum(nextdf[nextdf['county']==thisarea]['incidence'])
                nextcounty = nextdf[nextdf['county']==thisarea]['county'].values[0]
            else:
                nextincidence=0  
                nextcounty = 'no Data'
        except:
            nextcount=0
            nextincidence=0  
            nextcounty = 'no Data'
        LabelDF=LabelDF.append({'count':nextcount,'incidence':nextincidence,'county':nextcounty}, ignore_index=True)
        if (w % 1000 == 0):
            print(w)
    return LabelDF
        
label = FindLabels(newDF)
label.to_csv('StomachOutput.csv')

# Load Processed Data

In [3]:
Labels=pd.read_csv('StomachOutput.csv',encoding='latin-1').set_index('Unnamed: 0')
Data=pd.read_csv('StomachVirusInput.csv',encoding='latin-1').set_index('Unnamed: 0')
Data = Data[['week','year','count','incidence','X','Y','county','BinX','BinY']]
dfboth = Data.join(Labels,lsuffix='x',how='outer').dropna()
Data = dfboth[['week','year','countx','incidencex','X','Y','countyx','BinX','BinY']].rename(index=str, columns={"countx": "count", "incidencex": "incidence"})
Labels = dfboth[['count','incidence']]

In [4]:
allMaps = []
for m in range(15):
    for n in range(52):
        year = 2001+m
        thisweek = Data[Data['year']==year][Data[Data['year']==year]['week']==n]
        thisweek = thisweek.reset_index(drop=True)
        currentMap = np.zeros((16,16))
        currentCount = np.zeros((16,16))
        for i in range(shape(thisweek)[0]):
            inc = thisweek['incidence'][i]
            x = int(thisweek['BinX'][i][1:-1])
            y = int(thisweek['BinY'][i][1:-1])
            currentMap[x][y]=currentMap[x][y]+inc
            currentCount[x][y]=currentCount[x][y]+1
        currentMap = currentMap/currentCount
        allMaps.append(np.nan_to_num(currentMap))

C:\Users\Vivi\Anaconda3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


## Normalize Data

In [ ]:
import sklearn
from sklearn.preprocessing import normalize
normalized = newDF
"""x = normalized['BinX']
normBinX =sklearn.preprocessing.normalize([x],axis=1)
normalized['BinX']=normBinX[0]
x = normalized['BinY']
normBinY =sklearn.preprocessing.normalize([x])
normalized['BinY']=normBinY[0]
x = normalized['X']
normX =sklearn.preprocessing.normalize([x])
normalized['X']=normX[0]
x = normalized['Y']
normY =sklearn.preprocessing.normalize([x])
normalized['Y']=normY[0]"""
x = normalized['count']
normcount =sklearn.preprocessing.normalize([x])
normalized['count']=normcount[0]
x = normalized['incidence']
normInc =sklearn.preprocessing.normalize([x])
normalized['incidence']=normInc[0]

In [ ]:
normLabels = Labels
x = normLabels['count']
normcount =sklearn.preprocessing.normalize([x])
normLabels['count']=normcount[0]
x = normLabels['incidence']
normInc =sklearn.preprocessing.normalize([x])
normLabels['incidence']=normcount[0]

In [5]:
lenall = Data.shape[0]
trainSize = lenall//3*2#define sizes for data sets (train: 2/3, val:1/6, test:1/6 of whole dataset) 
valSize = 0#lenall//6

#Split data into train, val and test set------------------------------------------
trainingData = Data.iloc[0:trainSize].reset_index(drop=True)
trainingLabels = Labels.iloc[0:trainSize].reset_index(drop=True)
validationData = Data.iloc[trainSize+1:trainSize+valSize].reset_index(drop=True)
validationLabels = Labels.iloc[trainSize+1:trainSize+valSize].reset_index(drop=True)
testData = Data.iloc[trainSize+valSize+1:].reset_index(drop=True)
testLabels = Labels.iloc[trainSize+valSize+1:].reset_index(drop=True)

In [6]:
#Method to get batches------------------------------------------------------------
def getBatches(batchsize,typeBatch):
    if typeBatch=="train":
        numBatches = shape(trainingData)[0]//batchsize
        Batches=[]
        for count in range(numBatches-1):
            batch = trainingData[count*batchsize:(count+1)*batchsize]
            labels = trainingLabels[count*batchsize:(count+1)*batchsize]
            b,l=formatBatch(batch,labels,batchsize)
            newb = addGlobalCNN(b,batch.reset_index(),batchsize,batch['week'].reset_index(),batch['year'].reset_index())
            #print(newb)
            Batches.append([newb,l])
    elif typeBatch=="val":
        batch = validationData.sample(batchsize)
        labels = validationLabels.iloc[batch.index]
        b,l=formatBatch(batch,labels,batchsize)
        Batches  = [b,l]
    elif typeBatch=="test":
        batch = testData.sample(batchsize)
        labels = testLabels.iloc[batch.index]
        b,l=formatBatch(batch,labels,batchsize)
        Batches  = [b,l]
    else:
        print("typeBatch needs to be set to one of these values: train, val, test.")
        pass
    return Batches
def getLastWeek(thisweek,thisyear):
    weekdata = Data[Data['year']==thisyear][Data[Data['year']==thisyear]['week']==thisweek]
    #print(weekdata)
    size = shape(weekdata)[0]
    batch = formatBatch(weekdata,weekdata,size)
    return batch,size,weekdata['countyx']

def formatBatch(batch,labels,batchsize):
    batch = batch.reset_index(drop=True)
    labels = labels.reset_index(drop=True)
    newl = []
    newb = []
    for i,b in enumerate(batch['week']):
        x = batch['X'][i][1:-1]
        y = batch['Y'][i][1:-1]
        try:
            newb = np.append(newb,[int(b),int(batch['count'][i])/10,(batch['incidence'][i])/100,float(x)/100,float(y)/100])
        except:
            newb = np.append(newb,[int(b),int(batch['count'][i])/10,(batch['incidence'][i])/100,float(0)/100,float(0)/100])
    for ind,l in enumerate(labels['count']):
        newl = np.append(newl,[l/10,(labels['incidence'][ind])/100],axis=0)
    return np.reshape(newb,(batchsize,5)),np.reshape(newl,(batchsize,2))
def addGlobalCNN(batch,dfBatch,batchsize,week,year):
    records = []
    newb = []
    for i in range(batchsize):
        thisweek = week['week'][i]
        thisyear = year['year'][i]
        MapNum = (thisyear-2001)*52+thisweek
        x = dfBatch['BinX'][i]
        y = dfBatch['BinY'][i]
        binrisk = allMaps[MapNum][int(x[1:-1])][int(y[1:-1])]
        newb = np.append(batch[i],binrisk/100)
        records.append(newb)
    return records

In [ ]:
b = getBatches(50,'train')
#print(b)
for e in b:
    print("batch")
    print(e[0])
    print("labels")
    print(e[1])

In [8]:
tf.reset_default_graph()
%matplotlib notebook
learning_rate = 0.05
batch_size = 20
training_epochs = 100
display_step = 1
complexInput=False
n_hidden_1 = 250 # 1st layer number of neurons
n_hidden_2 = 250 # 2nd layer number of neurons
n_input = 6 
n_classes = 2 
step = 1

# tf Graph input
X = tf.placeholder("float32", [None,n_input])
Y = tf.placeholder("float32",[None,n_classes])

keep_probability = tf.placeholder("float")

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],stddev=1/n_input)),#**(-1/2))),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],stddev=1/n_hidden_1)),#**(-1/2))),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes],stddev=1/n_hidden_2))#**(-1/2)))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'bout': tf.Variable(tf.random_normal([n_classes]))
}

def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(x, weights['h1']), biases['b1'])),keep_probability)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.nn.dropout(tf.nn.tanh(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])),keep_probability)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['bout']
    return out_layer

outP = multilayer_perceptron(X)

loss_op = tf.reduce_mean(tf.losses.mean_squared_error(Y,outP))
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

#tf.summary.scalar("loss", loss_op)
#tf.summary.scalar("sample output",(outP[0][0])*10)
#tf.summary.scalar("sample target output",(Y[0][1])*100)

#merged_summaries = tf.summary.merge_all()

#train_writer = tf.summary.FileWriter("./summaries/trainFFNAreaRisk", tf.get_default_graph())
#validation_writer = tf.summary.FileWriter("./summaries/validationFFNAreaRisk")

In [30]:
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    saver = tf.train.Saver()
    kp=1.0
    samplePred = []
    sample = []
    allLoss = []
    for n in range(training_epochs):
        losses = []
        for b in getBatches(batch_size,"train"):
            _,loss,output = sess.run([train_op,loss_op,outP], feed_dict={X: b[0],Y: b[1],keep_probability:kp})  

            losses = np.append(losses,loss)
            step = step + 1
            #train_writer.add_summary(_summaries, step)
            #if (step%1000==0):
             #   saver.save(sess,"./log/FFNCountyRisk.ckpt")
        #b = getBatches(batch_size,'val')
        #_,Valloss,output,_Valsummaries = sess.run([train_op,loss_op,outP,merged_summaries], feed_dict={X: np.reshape(b[0],(batch_size,5)),Y: np.reshape(b[1],(batch_size,2)),keep_probability:kp})  
        #print("validation Loss: ",Valloss)
        #validation_writer.add_summary(_Valsummaries, step)
        print(output)
        lastout = output[3]
        samplePred = np.append(samplePred,output)#lastout[1])
        sample = np.append(sample,b[1])
        allLoss = np.append(allLoss,np.mean(losses))
        print("epoch: ",n,' - Step: ',step)
        print("pred: ",lastout)
        print("true: ",b[1][2])
        print("loss: ",np.mean(losses))
        #print(losses[0])

[[0.28091413 0.7750211 ]
 [0.30363613 0.6703932 ]
 [0.29868233 0.6039038 ]
 [0.34543115 0.764542  ]
 [0.41057765 0.78798497]
 [0.47739643 0.7706523 ]
 [0.40196118 0.62850547]
 [0.34198827 0.5970838 ]
 [0.47197253 0.66380215]
 [0.40195996 0.72110975]
 [0.41442448 0.7231797 ]
 [0.32539755 0.5979036 ]
 [0.4166556  0.7860575 ]
 [0.35641596 0.621657  ]
 [0.33570763 0.61365235]
 [0.40221646 0.6691686 ]
 [0.8882832  0.9330362 ]
 [0.52288616 0.6596755 ]
 [0.3280705  0.59353745]
 [0.38995954 0.6856353 ]]
epoch:  0  - Step:  211651
pred:  [0.34543115 0.764542  ]
true:  [0.2        0.70842666]
loss:  0.1522153963272163


KeyboardInterrupt: 

In [32]:
#sp = samplePred
#s = sample
print(shape(s)[0]/23)
for i in range(int(shape(sp)[0]/(batch_size*2))):
    print(i)
    print(sp[i*2])
    #print(s[i*2])
    print(sp[i*2+1])
    #print(s[i*2+1])

40.0
0
0.27724987268447876
0.7669916152954102
1
0.3039957880973816
0.6681524515151978
2
0.3098495602607727
0.6049075722694397
3
0.34203600883483887
0.7560332417488098
4
0.39929091930389404
0.7782647013664246
5
0.4647103548049927
0.7637075185775757
6
0.402984619140625
0.6261228919029236
7
0.34808260202407837
0.597250759601593
8
0.4682995676994324
0.6587813496589661
9
0.39255183935165405
0.7144127488136292
10
0.4071752429008484
0.7187098860740662
11
0.3327401876449585
0.5982867479324341
12
0.4125635027885437
0.774476170539856
13
0.3607600927352905
0.6215968728065491
14
0.34033429622650146
0.6137465238571167
15
0.3987804055213928
0.6640257239341736
16
0.8928439617156982
0.920444905757904
17
0.5172681212425232
0.6526880860328674
18
0.33602553606033325
0.5938740968704224
19
0.3906683325767517
0.6805176138877869
20
0.2687119245529175
0.7584760785102844
21
0.2911517024040222
0.662869930267334
22
0.3020186424255371
0.5922251343727112


In [22]:
shape(samplePred)[0]/(batch_size*2)

23.0

# Make New Predictions:

In [ ]:
thisweek = 52
thisyear = 2016
with tf.Session() as sess:
    init = tf.initialize_all_variables()
    sess.run(init)
    saver = tf.train.Saver()
    kp=1.0
    saver.restore(sess, "C:/Users/Vivi/Desktop/hack4health_RKI-data-scripts/DieInfluenza/log/FFNCountyRisk.ckpt")
    print("model loaded")
    #b,s,c = getLastWeek(thisweek,thisyear)
    batch_size = s
    _,loss,output= sess.run([train_op,loss_op,outP], feed_dict={X: newinput,Y: b[1],keep_probability:kp}) 
    print(output)


In [ ]:
predicted = pd.DataFrame(np.multiply(output,100))
counties = pd.DataFrame(c)

d = predicted.join(counties.reset_index(),lsuffix='x')
#d = d[['countyx','1']]
d.columns = ['count','incidence','ID','county']
d['week']=1#thisweek+1
d['year']=thisyear+1
d = d[['count','incidence','county','week','year']]
d.to_csv('Prediction_1_2017.csv')

In [ ]:
mses = []
for b in getBatches(batch_size,'train'):
    A = [b[0][0][1],b[0][0][2]]
    B = b[1][0]
    mse = ((A-B)**2).mean()
    mses.append(mse)